In [1]:
import pandas as pd

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
tp_df = pd.read_csv("dataset/train_true_positive_features.csv")
tn_df = pd.read_csv("dataset/train_true_negative_features.csv")
df = pd.concat([tp_df, tn_df])
df = df.reset_index(drop = True)

In [4]:
df = df[['vgg_cosine', 'vgg_euclidean_l2'
         , 'facenet_cosine', 'facenet_euclidean_l2'
         , 'openface_cosine', 'openface_euclidean_l2'
         , 'is_related']]

In [5]:
df.sample(5)

,vgg_cosine,vgg_euclidean_l2,facenet_cosine,facenet_euclidean_l2,openface_cosine,openface_euclidean_l2,is_related
101280,0.502680,1.002676,0.678233,1.164675,1.178907,1.535517,1
358811,0.529600,1.029175,0.819899,1.280546,0.645098,1.135868,0
125682,0.586000,1.082590,0.721757,1.201464,0.964883,1.389160,1
152342,0.434440,0.932137,0.399840,0.894248,0.302438,0.777737,1
100969,0.412297,0.908071,0.600329,1.095746,0.386677,0.879406,1


# Train Test Split

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_test = train_test_split(df, test_size=0.15, random_state=17)

# H2O AutoML

In [8]:
import h2o
from h2o.automl import H2OAutoML

In [9]:
h2o.__version__

'3.26.0.3'

In [10]:
# specify maximum memory size and number of threads based on your system

In [11]:
import multiprocessing
multiprocessing.cpu_count()

72

In [12]:
import psutil
psutil.virtual_memory()

svmem(total=1622764929024, available=669890707456, percent=58.7, used=949029715968, free=574988288000, active=933365055488, inactive=94913318912, buffers=0, cached=98746925056, shared=961851392, slab=2740187136)

In [13]:
#h2o.init()
h2o.init(ip="127.0.0.1", max_mem_size_GB = 200, nthreads = 10)

Checking whether there is an H2O instance running at http://127.0.0.1:54321 . connected.


H2O cluster uptime:,6 mins 51 secs
H2O cluster timezone:,Europe/Istanbul
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.3
H2O cluster version age:,11 days
H2O cluster name:,H2O_from_python_sefik_xmj4a9
H2O cluster total nodes:,1
H2O cluster free memory:,177.0 Gb
H2O cluster total cores:,72
H2O cluster allowed cores:,10
H2O cluster status:,"locked, healthy"


In [14]:
#pandas to h2o frame
hf = h2o.H2OFrame(x_train)

#alternatively, we can directly import h2o frame instead of converting from pandas
#hf = h2o.import_file('dataset/x_train.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [15]:
hf.head()

vgg_cosine,vgg_euclidean_l2,facenet_cosine,facenet_euclidean_l2,openface_cosine,openface_euclidean_l2,is_related
0.461026,0.960236,0.759346,1.23235,0.380667,0.872545,1
0.523298,1.02303,0.820089,1.28069,0.572227,1.06979,1
0.57931,1.07639,0.834568,1.29195,0.411612,0.907317,1
0.661336,1.15007,1.25314,1.58312,0.809752,1.2726,0
0.380073,0.871864,0.419412,0.915873,0.386296,0.878972,1
0.586928,1.08345,0.980849,1.40061,0.791823,1.25843,0
0.543978,1.04305,0.900194,1.34179,0.88473,1.33021,1
0.378311,0.86984,0.799367,1.26441,0.467552,0.967008,0
0.483638,0.983502,0.590177,1.08644,0.411636,0.907343,1
0.874395,1.32242,0.935428,1.36779,1.11803,1.49535,0


In [16]:
y_label = "is_related"
x_labels = list(df.drop(columns=[y_label]).columns)

In [17]:
x_labels

['vgg_cosine',
 'vgg_euclidean_l2',
 'facenet_cosine',
 'facenet_euclidean_l2',
 'openface_cosine',
 'openface_euclidean_l2']

In [18]:
#this is a binary classification problem. convert is_related column to enum type instead of numerical
#otherwise, it would be a regression problem
hf[y_label] = hf[y_label].asfactor()

In [19]:
#hf.describe()

In [20]:
aml = H2OAutoML(max_runtime_secs=60*60*2)

In [21]:
aml.train(x = x_labels, y = y_label, training_frame = hf)

AutoML progress: |████████████████████████████████████████████████████████| 100%


# Leaderboard

In [22]:
lb = aml.leaderboard

In [26]:
#lb.head()
lb.head(rows=lb.nrows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20190904_051328,0.724967,0.581732,0.341478,0.445439,0.198416
StackedEnsemble_AllModels_AutoML_20190904_051328,0.724951,0.581743,0.344451,0.445446,0.198422
XGBoost_grid_1_AutoML_20190904_051328_model_6,0.724634,0.581235,0.344413,0.445333,0.198321
XGBoost_grid_1_AutoML_20190904_051328_model_7,0.724633,0.581262,0.343124,0.445342,0.19833
XGBoost_3_AutoML_20190904_051328,0.724627,0.581256,0.343848,0.44534,0.198327
XGBoost_grid_1_AutoML_20190904_051328_model_11,0.724596,0.581288,0.344023,0.445352,0.198339
GBM_grid_1_AutoML_20190904_051328_model_3,0.724551,0.581298,0.34251,0.445363,0.198348
XGBoost_grid_1_AutoML_20190904_051328_model_8,0.724526,0.58139,0.343837,0.445379,0.198362
XGBoost_grid_1_AutoML_20190904_051328_model_12,0.724495,0.581425,0.343161,0.445397,0.198378
GBM_1_AutoML_20190904_051328,0.724479,0.581346,0.344121,0.445382,0.198365


In [24]:
saved_model = h2o.save_model(aml.leader, path = "", force=True)

In [25]:
saved_model

'/outputs/sefik/kinship/StackedEnsemble_BestOfFamily_AutoML_20190904_051328'

In [27]:
#you can directly load the best model by running the load_model command
#aml = h2o.load_model("/outputs/sefik/kinship/StackedEnsemble_BestOfFamily_AutoML_20190904_051328")

# Validation Score

In [28]:
hf_val = h2o.H2OFrame(x_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [29]:
perf = aml.model_performance(hf_val)

In [30]:
perf.auc()

0.7264606340321227


In [31]:
perf.accuracy() [0][1]

0.697363336404982
